# Python API examples


## Creates

- createUser
- createMarket
- createTrade

### Create user

Creates a new user. Returns signature and verify keys.

~~~~
import requests
url = 'https://blocmarket.herokuapp.com/createUser'
headers = {'content-type': 'application/json'}
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)
~~~~

Example return as json:

~~~~
{'signingKey': '0cca0a3739eba76cc78823d9f6e898379014d8c53172f5e45c171a3c54a9f477',
 'traderId': '1',
 'verifyKey': 'cdcfb59431b2579a681cee65656cbed6f8d578d5cc30d3e759e86c1d3e3529ef'}
~~~~
        
### Create/update market

Creates/updates a market. Returns True and market details if successful. 

Specify:
- Market root and branch id
- Minimum and maximum outcomes
- Trader Id
- Sign and verify keys

Note that sending the private key is a terrible idea. In production signing is done on client side.

~~~~
sk = '0cca0a3739eba76cc78823d9f6e898379014d8c53172f5e45c171a3c54a9f477'
vk = 'cdcfb59431b2579a681cee65656cbed6f8d578d5cc30d3e759e86c1d3e3529ef'
tId = 2

url = 'https://blocmarket.herokuapp.com/createMarket'
headers = {'content-type': 'application/json'}
content_makemarket = {"signingKey": sk,
                    "traderId": tId, 
                    "verifyKey": vk,
                    "marketRootId": 1, 
                    "marketBranchId": 1, 
                    "marketMin": 0,
                    "marketMax":1e4}

response = requests.post(url, data=json.dumps(content_makemarket), headers=headers)
~~~~

Example return as json:

~~~~
{'allChecks': "{'inputChk': True, 'marketLimitChk': True, 'traderIdChk': True, 'marketId': '1', 'marketRangeChk': True, 'marketIndChk': True, 'sigChk': True, 'chainChk': True, 'ownerChk': False, 'timeChk': True}",
 'checks': 'True',
 'marketBranchId': 1,
 'marketId': 1,
 'marketMax': 1e4,
 'marketMin': 0,
 'marketRootId': 1,
 'traderId': 2}

~~~~
    
### Create trade

Create a trade. Returns True and trade details if successful.

Specify:

- Trader Id
- Market Id
- Price
- Quantity
- Sign and verify keys

~~~~
url = 'https://blocmarket.herokuapp.com/createTrade'
headers = {'content-type': 'application/json'}
content_maketrade = {"signingKey": sk,
                     "traderId": int(tId),
                     "verifyKey": vk,
                     "marketId": mkId,
                     "price": 5510,
                     "quantity":1}
response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)



~~~~

Example return as json:
~~~~

{'allChecks': "{'inputChk': True, 'traderIdChk': True, 'marketChk': True, 'sigChk': True, 'chainChk': True, 'timeChk': True, 'colChk': True}",
 'checks': 'True',
 'marketId': 1,
 'price': 5510,
 'quantity': 1,
 'traderId': 1}


~~~~

## Views

- viewMarketBounds
- viewOrderBook
- viewOpenTrades
- viewMatchedTrades

### View markets

~~~~
url = 'https://blocmarket.herokuapp.com/viewMarketBounds'
headers = {'content-type': 'application/json'}
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)
~~~~

Example response as json:

~~~~
'{"marketId":{"0":1,"1":2,"2":3,"3":4},
"marketRootId":{"0":1,"1":2,"2":3,"3":4},
"marketBranchId":{"0":1,"1":1,"2":1,"3":1},
"marketMin":{"0":0.0,"1":0.0,"2":0.0,"3":0.0},
"marketMax":{"0":,"1":7000,"2":7000,"3":7000}}'
~~~~

### View full order book

~~~~
url = 'https://blocmarket.herokuapp.com/viewOrderBook'
headers = {'content-type': 'application/json'}
content = {'marketId': 1}
response = requests.post(url, data=json.dumps(content), headers=headers, stream=True)
~~~~

Example response as json:

~~~~
'{"marketId":{"0":1,"1":1},"price":{"0":5510,"1":5510},
"quantity":{"0":1.0,"1":-1.0,"traderId":{"0":1,"1":1},
"iMatched":{"0":true,"1":true,
"timeStampUTC":{"0":1547976585838,"1":1547976602766}}'
~~~~

Using a stream is only necessary if the order book is large and the request is truncated by the server, but it doesn't do any harm.

### View open trades

~~~~
url = 'https://https://blocmarket.herokuapp.com/viewOpenTrades'
content = {'marketId': 1}
response = requests.post(url, data=json.dumps(content), headers=headers, stream=True)
~~~~

~~~~
'{"marketId":{"12":1,"15":1,"18":1,"21":1,"24":1},
"price":{"12":4300,"15":4300,"18":4300,"21":4300,"24":4300},
"quantity":{"12":-1.0,"15":-1.0,"18":-1.0,"21":-1.0,"24":-1.0},
"traderId":{"12":1,"15":1,"18":1,"21":1,"24":1},"timeStampUTC":
{"12":1547976742476,"15":1547976750344,"18":1547976758683,"21":1547976768968,"24":1547976775537}}'
~~~~

### View matched trades

~~~~
url = 'https://blocmarket.herokuapp.com/viewMatchedTrades'
headers = {'content-type': 'application/json'}
content = {'marketId': 1}
response = requests.post(url, data=json.dumps(content), headers=headers, stream=True)
~~~~

Example response as json:

~~~~
'{"marketId":{"0":1,"1":1},"price":{"0":4300,"1":5500},"quantity":{"0":0.0,"1":0.0},"traderId":{"0":1,"1":1}}'

~~~~

### View trader summary

A view joining order book and marketTable for a particular trader.

~~~~
url = 'https://blocmarket.herokuapp.com/viewTradeSummary'
headers = {'content-type': 'application/json'}
content = {'traderId': 2}
response = requests.post(url, data=json.dumps(content), headers=headers, stream=True)
~~~~

Example response as json:

~~~~
'{"marketId":{"0":1,"1":1},"price":{"0":8000,"1":5000},
"quantity":{"0":-1.0,"1":-1.0},"traderId":{"0":2,"1":2},
"iMatched":{"0":false,"1":true},
"timeStampUTC":{"0":1547982939316,"1":1547982938964},
"marketMin":{"0":0.0,"1":0.0},"marketMax":{"0":1e4,"1":1e4}}'
~~~~


# Python demonstration

In [3]:
import requests
import json
import numpy as np
import pandas as pd
import time
import datetime

%matplotlib inline
import matplotlib.pyplot as plt

# Running flask app from powershell
# $env:FLASK_APP = "api"
# $env:FLASK_ENV = "development"
# flask run

# Running flask from terminal on OSX
# export FLASK_APP=api
# export FLASK_ENV=development
# flask run

# Running with gunicorn (heroku, digitalocean, etc):
# Procfile >
# web: gunicorn wsgi --log-file -
# wsgi.py >
# from api import app as application
#
#if __name__ == "__main__":
#    application.run()

# https://devcenter.heroku.com/articles/getting-started-with-python
# https://www.digitalocean.com/community/tutorials/how-to-serve-flask-applications-with-gunicorn-and-nginx-on-ubuntu-14-04


In [57]:
#baseurl = 'https://blocmarket.herokuapp.com/'
baseurl = 'http://127.0.0.1:5000/'
#baseurl = 'http://127.0.0.1:5000/'

In [58]:
# Create a new user
url = baseurl +'createUser'
headers = {'content-type': 'application/json'}
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)

In [59]:
# Copy keys and traderid from response
sk = response.json()['signingKey']
vk = response.json()['verifyKey']
tId = int(response.json()['traderId'])


In [60]:
vk

'7ec5cd731f7c2a97a31c15b6d12f56994f2c6701742e9660493db09d4cb74a52'

In [14]:
# Create a new market
mkRtId = 8
mkBrId = 1


url = baseurl + 'createMarket'
content_makemarket = {"signingKey": sk,
                        "traderId": tId, 
                        "verifyKey": vk,
                        "marketRootId": mkRtId, 
                        "marketBranchId": mkBrId, 
                        "marketMin": 1100,
                        "marketMax":7700}
# Post market
response = requests.post(url, data=json.dumps(content_makemarket), headers=headers)


In [16]:
# response = requests.post(url, data=json.dumps(content_makemarket), headers=headers)
# View response as json
response.json()

{'allChecks': "{'inputChk': True, 'marketLimitChk': True, 'traderIdChk': True, 'marketId': '8', 'marketRangeChk': True, 'marketIndChk': True, 'sigChk': True, 'chainChk': True, 'ownerChk': True, 'timeChk': True}",
 'checks': 'True',
 'marketBranchId': 1,
 'marketId': 8,
 'marketMax': 7700,
 'marketMin': 1100,
 'marketRootId': 8,
 'traderId': 3}

In [17]:
# Save market id from response
mkId = response.json()['marketId']
mkId

8

In [19]:
# Create a trade
url = baseurl+'createTrade'

content_maketrade = {"signingKey": sk,
                     "traderId": tId,
                     "verifyKey": vk,
                     "marketId": mkId,
                     "price": 5510,
                     "quantity":1}
response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)

In [20]:
# View response as json
response.json()

{'allChecks': "{'tradeId': 2766, 'inputChk': True, 'traderIdChk': True, 'marketChk': True, 'sigChk': True, 'chainChk': True, 'timeChk': True, 'colChk': True}",
 'checks': 'True',
 'marketId': 8,
 'price': 5510,
 'quantity': 1,
 'tradeId': 2766,
 'traderId': 3}

In [21]:
# View response as a dataframe
pd.DataFrame(response.json(), index=[0])


,allChecks,checks,marketId,price,quantity,tradeId,traderId
0,"{'tradeId': 2766, 'inputChk': True, 'traderIdC...",True,8,5510,1,2766,3


In [22]:
mId = response.json()['marketId']

In [23]:
# Create a matched trade
url = baseurl+'createTrade'
content_maketrade['quantity'] = -1
response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)

In [24]:
# View
response.json()

{'allChecks': "{'tradeId': 2767, 'inputChk': True, 'traderIdChk': True, 'marketChk': True, 'sigChk': True, 'chainChk': True, 'timeChk': True, 'colChk': True}",
 'checks': 'True',
 'marketId': 8,
 'price': 5510,
 'quantity': -1,
 'tradeId': 2767,
 'traderId': 3}

In [25]:
# Create many matched trades
url = baseurl+'createTrade'

for i in range(5):
    t = time.time()
    content_maketrade['quantity'] = -1
    response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)
    content_maketrade['quantity']=1
    response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)
    print('matched trade: ' + str(i) + ' in ' + str(time.time() - t) + ' seconds.')


matched trade: 0 in 0.1913449764251709 seconds.
matched trade: 1 in 0.16818618774414062 seconds.
matched trade: 2 in 0.1766986846923828 seconds.
matched trade: 3 in 0.17381906509399414 seconds.
matched trade: 4 in 0.1789078712463379 seconds.


In [26]:
content_maketrade

{'signingKey': '41affb8281a16a25789a156f24ae78d0e3722522a7c1a66c244a4dcae5fd144c',
 'traderId': 3,
 'verifyKey': '4f6788138e216c3f366327a28e475db3303425a0ff8ab437b2c3350d3f904274',
 'marketId': 8,
 'price': 5510,
 'quantity': 1}

In [27]:
# Create many markets
url = baseurl+'createMarket'

for i in range(10):
    t = time.time()

    content_makemarket = {"signingKey": sk,
      "traderId": int(tId), "verifyKey": vk,
      "marketRootId": i+1, "marketBranchId": 1, "marketMin": 0, "marketMax":70}

    response = requests.post(url, data=json.dumps(content_makemarket), headers=headers)
    if response.json()['checks'] == 'True':
        print('market created: ' + str(i+1) + ' in ' + str(time.time() - t) + ' seconds.')
    else:
        print('failed to create market ' +str(i+1))
    

failed to create market 1
failed to create market 2
failed to create market 3
failed to create market 4
failed to create market 5
failed to create market 6
failed to create market 7
failed to create market 8
failed to create market 9
failed to create market 10


In [28]:
response.json()

{'allChecks': "{'inputChk': True, 'marketLimitChk': False, 'traderIdChk': True, 'marketId': '9', 'marketRangeChk': True, 'marketIndChk': True, 'sigChk': True, 'chainChk': True, 'ownerChk': True, 'timeChk': True}",
 'checks': 'False',
 'marketBranchId': 1,
 'marketId': 9,
 'marketMax': 0.7,
 'marketMin': 0,
 'marketRootId': 10,
 'traderId': 3}

In [32]:
# Create many sub-markets
url = baseurl+'createMarket'
marketRootId = tId

for i in range(10):
    t = time.time()

    content_makemarket = {"signingKey": sk,
      "traderId": int(tId), "verifyKey": vk,
      "marketRootId": marketRootId, "marketBranchId": 1+i, "marketMin": 0, "marketMax":70}

    response = requests.post(url, data=json.dumps(content_makemarket), headers=headers)
    if response.json()['checks'] == 'True':
        print('market created: ' + str(i+1) + ' in ' + str(time.time() - t) + ' seconds.')
    else:
        print('failed to create sub market ' +str(i+1))
    

failed to create sub market 1
market created: 2 in 4.146286725997925 seconds.
market created: 3 in 4.077319860458374 seconds.
market created: 4 in 4.114520072937012 seconds.
market created: 5 in 4.151880979537964 seconds.
market created: 6 in 4.228972673416138 seconds.
market created: 7 in 4.198970079421997 seconds.
market created: 8 in 4.425649881362915 seconds.
market created: 9 in 4.187408924102783 seconds.
market created: 10 in 4.178578853607178 seconds.


In [33]:
response.json()

{'allChecks': "{'inputChk': True, 'marketLimitChk': True, 'traderIdChk': True, 'marketId': '17', 'marketRangeChk': True, 'marketIndChk': True, 'sigChk': True, 'chainChk': True, 'ownerChk': True, 'timeChk': True}",
 'checks': 'True',
 'marketBranchId': 10,
 'marketId': 17,
 'marketMax': 70,
 'marketMin': 0,
 'marketRootId': 3,
 'traderId': 3}

In [34]:
# View maximum and minimum of each market
url = baseurl+'viewMarketBounds'
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)


In [35]:
z = pd.read_json(response.json())
z.loc[z['marketId']==1,'marketDesc'][0]



'This is a market with a name.'

In [36]:
z = pd.read_json(response.json())
max(z['marketRootId'])


8

In [40]:
# View whole order book
url = baseurl+'viewOrderBook'
content = {'marketId': 8}
response = requests.post(url, data=json.dumps(content), headers=headers, stream=True)

In [41]:
pd.read_json(response.json())


,tradeId,marketId,price,quantity,traderId,iMatched,timeStampUTC
0,2768,8,5510,-1,3,True,2019-10-20 10:00:23.357
1,2766,8,5510,1,3,True,2019-10-20 09:58:26.250
10,2775,8,5510,1,3,True,2019-10-20 10:00:23.965
11,2776,8,5510,-1,3,True,2019-10-20 10:00:24.055
12,2777,8,5510,1,3,False,2019-10-20 10:00:24.144
2,2769,8,5510,1,3,True,2019-10-20 10:00:23.447
3,2770,8,5510,-1,3,True,2019-10-20 10:00:23.531
4,2771,8,5510,1,3,True,2019-10-20 10:00:23.615
5,2772,8,5510,-1,3,True,2019-10-20 10:00:23.702
6,2765,8,5510,1,3,True,2019-10-20 09:58:09.477


In [42]:
content_makemarket

{'signingKey': '41affb8281a16a25789a156f24ae78d0e3722522a7c1a66c244a4dcae5fd144c',
 'traderId': 3,
 'verifyKey': '4f6788138e216c3f366327a28e475db3303425a0ff8ab437b2c3350d3f904274',
 'marketRootId': 3,
 'marketBranchId': 10,
 'marketMin': 0,
 'marketMax': 70}

In [43]:
# Create many unmatched trades
url = baseurl+'createTrade'

for i in range(5):
    t = time.time()
    content_maketrade['quantity'] = -1
    content_maketrade['price'] = 4300
    response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)
    print('unmatched trade: ' + str(i) + ' in ' + str(time.time() - t) + ' seconds.')


unmatched trade: 0 in 0.10195374488830566 seconds.
unmatched trade: 1 in 0.0906069278717041 seconds.
unmatched trade: 2 in 0.08415627479553223 seconds.
unmatched trade: 3 in 0.08770513534545898 seconds.
unmatched trade: 4 in 0.08500099182128906 seconds.


In [45]:
a = response.json()
a

{'allChecks': "{'tradeId': 2782, 'inputChk': True, 'traderIdChk': True, 'marketChk': True, 'sigChk': True, 'chainChk': True, 'timeChk': True, 'colChk': True}",
 'checks': 'True',
 'marketId': 8,
 'price': 4300,
 'quantity': -1,
 'tradeId': 2782,
 'traderId': 3}

In [46]:
# View all open (unmatched trades)
url = baseurl+'viewOpenTrades'

content = {'marketId': 1}
response = requests.post(url, data=json.dumps(content), headers=headers)

In [32]:
pd.read_json(response.json())

,tradeId,marketId,price,quantity,traderId,timeStampUTC
12,13,1,4300,-1,1,2019-05-11 06:11:06.429
13,14,1,4300,-1,1,2019-05-11 06:11:06.519
14,15,1,4300,-1,1,2019-05-11 06:11:06.611
15,16,1,4300,-1,1,2019-05-11 06:11:06.701
16,17,1,4300,-1,1,2019-05-11 06:11:06.791


In [33]:
# View matched trades (summed over marketId/price/traderId)
url = baseurl+'viewMatchedTrades'
content = {'marketId': mkId}
response = requests.post(url, data=json.dumps(content), headers=headers, stream=True)

In [34]:
pd.read_json(response.json())

,marketId,price,quantity,traderId
0,1,5510,0,1


In [44]:
# Create many matched trades (own)
url = baseurl+'createTrade'

for i in range(5): 
    t = time.time()
    content_maketrade['quantity'] = -1
    response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)
    content_maketrade['quantity']=1
    response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)
    print('matched trade: ' + str(i) + ' in ' + str(time.time() - t) + ' seconds.')


matched trade: 0 in 0.19957184791564941 seconds.
matched trade: 1 in 0.18017983436584473 seconds.
matched trade: 2 in 0.1808781623840332 seconds.
matched trade: 3 in 0.18397116661071777 seconds.
matched trade: 4 in 0.17993497848510742 seconds.


In [47]:
# View summary of trader for a particular trader
url = baseurl+'viewTradeSummary'
content = {'traderId': int(1)}
response = requests.post(url, data=json.dumps(content), headers=headers, stream=True)

In [48]:
pd.read_json(response.json())

,tradeId,marketId,price,quantity,traderId,iMatched,timeStampUTC,marketRootId,marketBranchId,marketMin,marketMax,marketMinOutcome,marketMaxOutcome
0,2768,8,5510,-1,3,True,2019-10-20 10:00:23.357,8,1,1100,7700,4410,-2190
1,2766,8,5510,1,3,True,2019-10-20 09:58:26.250,8,1,1100,7700,-4410,2190
10,2775,8,5510,1,3,True,2019-10-20 10:00:23.965,8,1,1100,7700,-4410,2190
11,2776,8,5510,-1,3,True,2019-10-20 10:00:24.055,8,1,1100,7700,4410,-2190
12,2777,8,5510,1,3,False,2019-10-20 10:00:24.144,8,1,1100,7700,-4410,2190
13,2778,8,4300,-1,3,False,2019-10-20 10:07:01.798,8,1,1100,7700,3200,-3400
14,2779,8,4300,-1,3,False,2019-10-20 10:07:01.887,8,1,1100,7700,3200,-3400
15,2780,8,4300,-1,3,False,2019-10-20 10:07:01.976,8,1,1100,7700,3200,-3400
16,2781,8,4300,-1,3,False,2019-10-20 10:07:02.063,8,1,1100,7700,3200,-3400
17,2782,8,4300,-1,3,False,2019-10-20 10:07:02.148,8,1,1100,7700,3200,-3400


In [49]:
# Get a signed UTC timestamp
url = baseurl+'getSignedUTCTimestamp'
content = {}
response = requests.get(url, data=json.dumps(content), headers=headers)


In [50]:
response.json()

{'timeStampUTC': '2019-10-20 10:08:24.601437',
 'timeStampUTCSignature': "b'\\xcds\\xc9\\xf6JX\\x94y\\x13;\\x089h\\xf1\\xf8\\xaanC\\xe8\\xb4M\\xef\\xa4\\x8f\\xff\\x8b\\x02\\xdc\\xfa\\xbf\\xe5^\\xae\\x82\\xa0\\xe9X\\xa3Qe\\xd8\\xdf\\xd93c\\xa3\\xee\\xe9)\\x0f\\xd7;s\\xee\\xa0$~\\xec\\xaa\\x8f/\\xfa\\xb1\\x0b'",
 'verifyKey': 'cdcfb59431b2579a681cee65656cbed6f8d578d5cc30d3e759e86c1d3e3529ef'}

In [55]:
# View summary of trader for a particular trader
url = baseurl+'viewTickHistory'
content = {'marketId': int(1), 'startTime': datetime.datetime(2019,1,1).timestamp()*1000, 'endTime': 2e9*1000}
response = requests.post(url, data=json.dumps(content), headers=headers, stream=True)

In [56]:
pd.read_json(response.json())

,tickType,tradeId,xTradeId,marketId,price,quantity,traderId,iMatched,timeStampUTC
0,BID,2765,2767.0,8,5510,1,3,True,2019-10-20 09:58:09.477
1,BID,2766,2767.0,8,5510,1,3,True,2019-10-20 09:58:26.250
2,ASK,2778,NaN,8,4300,-1,3,False,2019-10-20 10:07:01.798
3,ASK,2779,NaN,8,4300,-1,3,False,2019-10-20 10:07:01.887
4,ASK,2780,NaN,8,4300,-1,3,False,2019-10-20 10:07:01.976
5,ASK,2781,NaN,8,4300,-1,3,False,2019-10-20 10:07:02.063
6,ASK,2782,NaN,8,4300,-1,3,False,2019-10-20 10:07:02.148



# Curl API examples



## Create user

~~~~
curl --header "Content-Type: application/json" -X POST http://blocmarket.herokuapp.com/createUser
~~~~
        
## Create market

~~~~
curl --data '{"signingKey_hex": "ece2efc138c8298d43caba1315ceda614e20644c74d46fed37871c47ea19afdf",
"traderId": 1, 
"verifyKey_hex": "9690a2e12971ae452d68bf3d08405090d45791533cf80740fd186aea4b6773fc",
"marketRootId": 1, 
"marketBranchId": 5, 
"marketMin": 0, 
"marketMax": 1e4}' 
--header "Content-Type: application/json" -X POST http://blocmarket.herokuapp.com/createMarket
~~~~
    
## Create trade

~~~~
curl --data '{"signingKey_hex": "ece2efc138c8298d43caba1315ceda614e20644c74d46fed37871c47ea19afdf",
"traderId": 1, 
"verifyKey_hex": "9690a2e12971ae452d68bf3d08405090d45791533cf80740fd186aea4b6773fc",
"marketId": 1,
"price": 1212,
"quantity": 1}' --header "Content-Type: application/json" -X POST http://blocmarket.herokuapp.com/createTrade

~~~~
        
## View order book (output to file)

~~~~
curl --header "Content-Type: application/json" -X POST http://blocmarket.herokuapp.com/viewOrderBook --output ob.txt
~~~~    
        
Also can load json from file with something like 
~~~~ 
curl -d "@data.json" -X POST http://blocmarket.herokuapp.com/data 
~~~~


# Matlab API examples


## Create user

~~~~
resp = webread('http://blocmarket.herokuapp.com/createUser')

~~~~

Returns something like

~~~~
'{"signingKey": '57f611e7cb7c5d4123be69663ccd06442c64db1a4f1161399dbb1b9bb6cc0e06'
           "traderId": '2'
      "verifyKey": 'ddaa5b0119a4ccfeffa36be0e6aac3682f798d021d5ded1d90512f2226e727b5'}'
~~~~

## Create market

~~~~
data = struct('signingKey', signingKey,...
'traderId', tId, ...
'verifyKey', verifyKey ,...
'marketRootId', 2, ...
'marketBranchId', 1, ...
'marketMin', 100, ...
'marketMax', 1e4)
resp = webwrite('http://blocmarket.herokuapp.com/createMarket', data, weboptions('RequestMethod', 'post','MediaType', 'application/json'))
~~~~

Returns something like

~~~~
'{
             "checks": 'True'
     "marketBranchId": 1
          "marketMax": 10000
          "marketMin": 100
       "marketRootId": 2
           "traderId": 1
}'
~~~~
## Create trade

~~~~
data = struct('signingKey', signingKey,...
'traderId', tId,...
'verifyKey', verifyKey ,...
'marketId', 1,...
'price', 5100,...
'quantity', 1)
resp = webwrite('http://blocmarket.herokuapp.com/createTrade', data, weboptions('MediaType', 'application/json'))
~~~~

Returns something like
~~~~
'{
       "checks": 'True'
     "marketId": 1
        "price": 5100
     "quantity": 1
     "traderId": 1
 '}S
~~~~

## View market bounds
~~~~
data = struct('marketId', 1)
resp = webwrite('http://blocmarket.herokuapp.com/viewMarketBounds', data, weboptions('RequestMethod', 'post','contentType', 'json', 'Timeout', 10))
~~~~

## View order book
~~~~
data = struct('marketId':1)
resp = webwrite('http://blocmarket.herokuapp.com/viewOrderBook', data, weboptions('RequestMethod', 'post','contentType', 'json'), 'Timeout', 10)
~~~~

## View trade summary

~~~~
data = struct('traderId',1);
resp = webwrite('http://blocmarket.herokuapp.com/viewTradeSummary', data, weboptions('RequestMethod', 'post','MediaType', 'application/json', 'Timeout', 10))
~~~~


# Powershell

TODO


    

test cases with mismatched traderId/verifyKey
views to specific market
javascript/powershell for api
enforce max decimals DONE
table based checkcollateral
put up on digitalocean

write up another layer of trading interface